In [1]:
from pluto_survey_tools import QUESTIONNAIRE
import pluto_survey_tools.stats as stats
from IPython.display import display_markdown

## Min-max ranges

### Global

Overview of the possible minimum and maximum scores for each question and section of the questionnaire.

In [2]:
display_markdown(f'# Questionnaire: {stats.score_range_questionnaire(QUESTIONNAIRE)}', raw=True)

for section in QUESTIONNAIRE.sections:
    display_markdown(f'## {section.title}: {stats.score_range_section(section)}', raw=True)
    for i, question in enumerate(section.questions):
        display_markdown(f'Q{i+1}: {stats.score_range_question(question)}', raw=True)

# Questionnaire: (-60.0, 40.0)

## Information About the Applicant: (-8.0, 8.0)

Q1: (-1.0, 1.0)

Q2: (-2.0, 1.0)

Q3: (-1.0, 2.0)

...

Q3: (-3.0, 2.0)

Q4: (-1.0, 2.0)

Q5: (-1.0, 1.0)

Q6: (-3.0, 0.0)

Q7: (-3.0, 5.0)

### By axis

In [3]:
questions = QUESTIONNAIRE.questions
x_q = [q for q in questions if q.impact_keys[0] == 'x']
x_ranges = [stats.score_range_question(q) for q in x_q]
x_min = sum([r[0] for r in x_ranges])
x_max = sum([r[1] for r in x_ranges])

y_q = [q for q in questions if q.impact_keys[0] == 'y']
y_ranges = [stats.score_range_question(q) for q in y_q]
y_min = sum([r[0] for r in y_ranges])
y_max = sum([r[1] for r in y_ranges])

In [4]:
display_markdown(f'## X ~ Risk: ({x_min}, {x_max})', raw=True)
display_markdown(f'## Y ~ Public Value: ({y_min}, {y_max})', raw=True)

## X ~ Risk: (-51.0, 23.0)

## Y ~ Public Value: (-9.0, 17.0)